In [1]:
class Food(object):
    def __init__(self, n, v, w):
        self.name = n
        self.value = v
        self.calories = w
    def getValue(self):
        return self.value
    def getCost(self):
        return self.calories
    def density(self):
        return self.getValue() / self.getCost()
    def __str__(self):
        return self.name + ': <' + str(self.value) + ", " + str(self.calories) + '>'


In [2]:
def buildMenu(names, values, calories):
    """
    names, values, calories lists of same length.
    name a list of strings
    values and calories lists of numbers
    returns list of Foods
    """
    menu = []
    for i in range(len(values)):
        menu.append(Food(names[i], values[i], calories[i]))
    return menu

In [3]:
def greedy(items, maxCost, keyFunction):
    """
    Assumes items a list, maxCost >= 0,
    keyFunction maps elements of items to numbers
    """
    itemsCopy = sorted(items, key = keyFunction, reverse = True)
    result = []
    totalValue, totalCost = 0.0, 0.0
    for i in range(len(itemsCopy)):
        if (totalCost + itemsCopy[i].getCost()) <= maxCost:
            result.append(itemsCopy[i])
            totalCost += itemsCopy[i].getCost()
            totalValue += itemsCopy[i].getValue()
    return (result, totalValue)

In [4]:
def testGreedy(items, constraint, keyFunction):
    taken, val = greedy(items, constraint, keyFunction)
    print('Total value of items taken =', val)
    for item in taken:
        print('    ', item)

In [5]:
def testGreedys(foods, maxUnits):
    print("Use greedy by value to allocate", maxUnits, 'calories')
    testGreedy(foods, maxUnits, Food.getValue)
    print('\nUse greedy by cost to allocate', maxUnits, 'calories')
    testGreedy(foods, maxUnits, lambda x: 1/Food.getCost(x))
    print('\nUse greedy bo density to allocate', maxUnits, 'calories')
    testGreedy(foods, maxUnits, Food.density)

In [17]:
def maxVal(toConsider, avail):
    if toConsider == [] or avail == 0:
        result = (0, ())
    elif toConsider[0].getCost() > avail:
        result = maxVal(toConsider[1:], avail)
    else:
        """
        Assumes toConsider a list of items, avail"""
        nextItem = toConsider[0]
        withVal, withToTake = maxVal(toConsider[1:], avail - nextItem.getCost())
        withVal += nextItem.getValue()
        withoutVal, withoutToTake = maxVal(toConsider[1:], avail)
        if withVal > withoutVal:
            result = (withVal, withToTake + (nextItem,))
        else:
            result = (withoutVal, withoutToTake)
    return result

In [18]:
def testMaxVal(foods, maxUnits, printItems = True):
    print('Use search tree to allocate', maxUnits, 'calories')
    val, taken = maxVal(foods, maxUnits)
    print("Total value of items taken =", val)
    if printItems:
        for item in taken:
            print('    ', item)

In [19]:
names = ['wine', 'beer', 'pizza', 'burger', 'fries', 'cola', 'apple', 'donut', 'cake']

In [20]:
values = [89, 90, 95, 100, 90, 79, 50, 10]

In [21]:
len(names)

9

In [22]:
len(values)

8

In [23]:
calories = [123, 154, 258, 354, 365, 150, 95, 195]

In [24]:
foods = buildMenu(names, values, calories)

In [25]:
testGreedys(foods, 1000)

Use greedy by value to allocate 1000 calories
Total value of items taken = 424.0
     burger: <100, 354>
     pizza: <95, 258>
     beer: <90, 154>
     wine: <89, 123>
     apple: <50, 95>

Use greedy by cost to allocate 1000 calories
Total value of items taken = 413.0
     apple: <50, 95>
     wine: <89, 123>
     cola: <79, 150>
     beer: <90, 154>
     donut: <10, 195>
     pizza: <95, 258>

Use greedy bo density to allocate 1000 calories
Total value of items taken = 413.0
     wine: <89, 123>
     beer: <90, 154>
     cola: <79, 150>
     apple: <50, 95>
     pizza: <95, 258>
     donut: <10, 195>


In [26]:
testMaxVal(foods, 750)

Use search tree to allocate 750 calories
Totak value of items taken = 353
     cola: <79, 150>
     pizza: <95, 258>
     beer: <90, 154>
     wine: <89, 123>


In [27]:
import random

In [28]:
def buildLargeMenu(numItems, maxVal, maxCost):
    items = []
    for i in range(numItems):
        items.append(Food(str(i), random.randint(1, maxVal), random.randint(1, maxCost)))
    return items

In [29]:
def fastFib(n, memo = {}):
    """Assumes n in an int >= 0, memo used only by recursive calls
    Returns Fibonacci of n"""
    if n == 0 or n == 1:
        return 1
    try:
        return memo[n]
    except KeyError:
        result = fastFib(n-1, memo) + fastFib(n-2, memo)
        memo[n] = result
        return result

In [30]:
def fastMaxVal(toConsider, avail, memo = {}):
    if (len(toConsider), avail) in memo:
        result = memo[(len(toConsider), avail)]
    elif toConsider == [] or avail == 0:
        result = (0, ())
    elif toConsider[0].getCost() > avail:
        result = fastMaxVal(toConsider[1:], avail, memo)
    else:
        nextItem = toConsider[0]
        withVal, withToTake = fastMaxVal(toConsider[1:], avail - nextItem.getCost(), memo)
        withVal += nextItem.getValue()
        withoutVal, withoutToTake = fastMaxVal(toConsider[1:], avail, memo)
        
        # Choose better branch
        if withVal > withoutVal:
            result = (withVal, withToTake + (nextItem,))
        else:
            result = (withoutVal, withoutToTake)
    memo[(len(toConsider), avail)] = result
    return result

In [32]:
def testMaxVal(foods, maxUnits, algorithm, printItems = True):
    print("Menu contains", len(foods), 'items')
    print("Use search tree to allocate", maxUnits, 'calories')
    val, taken = algorithm(foods, maxUnits)
    if printItems:
        print("Total value of items taken =", val)
        for item in taken:
            print('    ', item)

In [33]:
for numItems in (5, 10, 15, 20, 25, 30, 35, 40, 45, 50):
    items = buildLargeMenu(numItems, 90, 250)
    testMaxVal(items, 750, fastMaxVal, True)

Menu contains 5 items
Use search tree to allocate 750 calories
Total value of items taken = 234
     4: <78, 175>
     3: <8, 27>
     2: <76, 40>
     1: <25, 229>
     0: <47, 158>
Menu contains 10 items
Use search tree to allocate 750 calories
Total value of items taken = 336
     9: <51, 83>
     8: <52, 104>
     6: <17, 85>
     5: <59, 92>
     3: <60, 108>
     2: <13, 14>
     1: <30, 108>
     0: <54, 149>
Menu contains 15 items
Use search tree to allocate 750 calories
Total value of items taken = 566
     9: <51, 83>
     12: <35, 45>
     10: <89, 124>
     8: <22, 8>
     6: <84, 12>
     5: <62, 118>
     4: <89, 7>
     2: <84, 204>
     1: <50, 93>
Menu contains 20 items
Use search tree to allocate 750 calories
Total value of items taken = 594
     9: <51, 83>
     8: <52, 104>
     12: <35, 45>
     10: <89, 124>
     8: <22, 8>
     6: <84, 12>
     5: <62, 118>
     4: <89, 7>
     7: <46, 25>
     1: <64, 168>
Menu contains 25 items
Use search tree to allocate 750 c